In [1]:
! pip install google-generativeai
! pip install python-dotenv
! pip install langchain
! pip install faiss-cpu
! pip install PyPDF2
! pip install sentence-transformers

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Using cached pypdf2-3.0.1-py3-none-any.whl.metadata (6.8 kB)
Using cached pypdf2-3.0.1-py3-none-any.whl (232 kB)
Defaulting to user installation because normal site-packages is not writeable


In [ ]:
import os
from dotenv import load_dotenv
import google.generativeai as genai
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
import gradio as gr

# Load environment variables
load_dotenv()
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

# Configure Gemini
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel('gemini-pro')

# PDF Processing Function
def process_telugu_pdf(pdf_path):
    loader = PyPDFLoader(pdf_path)
    pages = loader.load()
    
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200
    )
    chunks = text_splitter.split_documents(pages)
    
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
    vector_store = FAISS.from_documents(chunks, embeddings)
    
    return vector_store

# Load and process the Telugu PDF
pdf_path = r"C:\Users\jogin\OneDrive\Documents\GenAI\notebooks\user_projects\coconut.pdf"
vector_store = process_telugu_pdf(pdf_path)

def get_coconut_farming_response(query, history):
    # First, detect and translate Telugu query if needed
    translation_prompt = f"""Detect if this text is in Telugu. If it is Telugu, translate it to English. If it's already in English, return it as-is:
    Text: {query}"""
    
    translation_response = model.generate_content(translation_prompt)
    translated_query = translation_response.text
    
    # Get relevant context and generate response
    relevant_docs = vector_store.similarity_search(translated_query, k=3)
    context = "\n".join([doc.page_content for doc in relevant_docs])
    
    prompt = f"""You are a knowledgeable coconut farming expert. Using the context provided and your expertise, provide detailed practical advice.
    
    Context from coconut farming guide: {context}
    
    Question: {translated_query}
    
    Important: Translate your response to Telugu and format it properly. Ensure the Telugu text is clear and well-structured."""
    
    response = model.generate_content(prompt)
    return response.text

# Create Gradio Interface
def create_gradio_interface():
    with gr.Blocks(title="Coconut Farming Assistant") as demo:
        gr.Markdown("# 🌴 కొబ్బరి వ్యవసాయ సహాయకుడు")
        gr.Markdown("కొబ్బరి సాగు గురించి ఏవైనా ప్రశ్నలు అడగండి మరియు నిపుణుల సమాధానాలు పొందండి!")
        
        chatbot = gr.Chatbot(
            show_label=False,
            container=True,
            height=400
        )
        
        with gr.Row():
            msg = gr.Textbox(
                label="మీ ప్రశ్నను ఇక్కడ టైప్ చేయండి",
                placeholder="కొబ్బరి సాగు గురించి అడగండి...",
                show_label=False
            )
            submit = gr.Button("పంపండి", variant="primary")
        
        gr.Markdown("### ఉదాహరణలు:")
        gr.Examples(
            examples=[
                "కొబ్బరి సాగుకు ఉత్తమమైన నేల పరిస్థితులు ఏమిటి?",
                "కొబ్బరి చెట్లకు ఎంత తరచుగా నీరు పెట్టాలి?",
                "కొబ్బరి చెట్లలో సాధారణ వ్యాధులు ఏమిటి?",
                "కొబ్బరి దిగుబడిని ఎలా పెంచాలి?"
            ],
            inputs=msg
        )
        
        def respond(message, chat_history):
            bot_message = get_coconut_farming_response(message, chat_history)
            chat_history.append((message, bot_message))
            return "", chat_history
        
        submit.click(respond, [msg, chatbot], [msg, chatbot])
        msg.submit(respond, [msg, chatbot], [msg, chatbot])
    
    return demo

# Launch the interface
if __name__ == "__main__":
    demo = create_gradio_interface()
    demo.launch(share=True, debug=True)


c:\ProgramData\anaconda3\envs\GenAI\Lib\site-packages\gradio\components\chatbot.py:282: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860

Could not create share link. Missing file: c:\ProgramData\anaconda3\envs\GenAI\Lib\site-packages\gradio\frpc_windows_amd64_v0.3. 

Please check your internet connection. This can happen if your antivirus software blocks the download of this file. You can install manually by following these steps: 

1. Download this file: https://cdn-media.huggingface.co/frpc-gradio-0.3/frpc_windows_amd64.exe
2. Rename the downloaded file to: frpc_windows_amd64_v0.3
3. Move the file to this location: c:\ProgramData\anaconda3\envs\GenAI\Lib\site-packages\gradio


In [7]:
! pip install googletrans==3.1.0a0
! pip install google-cloud-translate



Defaulting to user installation because normal site-packages is not writeable
  Using cached httpx-0.13.3-py3-none-any.whl.metadata (25 kB)
  Using cached httpcore-0.9.1-py3-none-any.whl.metadata (4.6 kB)
  Using cached h11-0.9.0-py2.py3-none-any.whl.metadata (8.1 kB)
Using cached httpx-0.13.3-py3-none-any.whl (55 kB)
Using cached httpcore-0.9.1-py3-none-any.whl (42 kB)
Using cached h11-0.9.0-py2.py3-none-any.whl (53 kB)
  Attempting uninstall: h11
    Found existing installation: h11 0.14.0
    Uninstalling h11-0.14.0:
      Successfully uninstalled h11-0.14.0
  Attempting uninstall: httpcore
    Found existing installation: httpcore 1.0.7
    Uninstalling httpcore-1.0.7:
      Successfully uninstalled httpcore-1.0.7
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langsmith 0.3.8 requires httpx<1,>=0.23.0, but you have httpx 0.13.3 which is incompatible.
openai 1.61.1 requires httpx<1,>=0.23.0, but you have httpx 0.13.3 which is incompatible.
gradio 5.15.0 requires httpx>=0.24.1, but you have httpx 0.13.3 which is incompatible.
gradio-client 1.7.0 requires httpx>=0.24.1, but you have httpx 0.13.3 which is incompatible.


Defaulting to user installation because normal site-packages is not writeable
